In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from scipy import stats
import pingouin as pg

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from scipy import stats
import pingouin as pg

## Costo de seguros

El objetivo del ejercicio es realizar un análisis estadístico para identificar la relación del costo de un seguro y algunas variables de interés, como son edad, sexo, condición de fumador, número de hijos y región.

Este análisis se compone de dos partes:

1. **Análisis descriptivo**: Buscamos identificar el comportamiento del costo del seguro respecto a las demás variables, que genere conclusiones sobre la población de estudio, además de identificar relaciones entre las variables que nos permita construir un modelo de regresión.
$\newline$
2. **Análisis inferencial**: Buscamos construir un modelo de regresión para revisar el efecto de las variables edad, sexo, condición de fumador, número de hijos y región sobre el costo del seguro, y reforzar con ello el análisis descriptivo. Además, buscamos generar un modelo que nos permita estimar el costo del seguro en función de las características ya mencionadas.

### Descripción de los datos

Se cuenta con una tabla que contiene información sobre costos de un seguro médico, y algunas características correlacionadas con el costo del seguro: edad, sexo, índice de masa corporal, número de hijos, condición de fumador y región. En el siguiente bloque de código se leen los datos, y se realiza una inspección de la estructura de los datos.

In [6]:
## Leemos los datos con la función read_csv
df_seguros = pd.read_csv('insurance.csv')
print('Vista previa de los datos')
display(df_seguros.sample(3))
print('')

## Imprimimos el número de filas y de columnas de la tabla
print('Estructura de los datos:')
print('Filas   :', df_seguros.shape[0])
print('Columnas:', df_seguros.shape[1])
print('')

## Obtenemos el número de NAs en cada columna
print('Número de NAs por columna:')
display(pd.DataFrame(df_seguros.isna().sum()).transpose())

Vista previa de los datos


,edad,sexo,imc,hijos,fumador,region,cargos
74,44,masculino,27.40,2,0,suroeste,7726.85400
1331,23,femenino,33.40,0,0,suroeste,10795.93733
853,53,femenino,23.75,2,0,noreste,11729.67950



Estructura de los datos:
Filas   : 1338
Columnas: 7

Número de NAs por columna:


,edad,sexo,imc,hijos,fumador,region,cargos
0,0,0,0,0,0,0,0


Se tiene información de 1338 individuos. No tenemos información nula o NAs, por lo cual no tendremos que realizar ninguna eliminación de registros o imputación de valores.

### Análisis descriptivo

Para revisar el comportamiento de los datos, y otras variables de interés, utilizaremos algunos gráficos, como histogramas, boxplots o scatterplots. Revisaremos primero el comportamiento de la variable de interés, **cargos por seguro** y luego indagaremos en su relación con las demás variables.

#### Distribución de la variable cargos

La distribución de la variable **cargos** se puede asumir log-normal (propia de las variables económicas) o incluso se puede suponer cierta normalidad, sin embargo para ello podría ser útil utilizar alguna transformación. Para el ejercicio, asumiremos que sigue una distribución normal.

In [7]:
px.histogram(df_seguros, x = 'cargos', nbins=40, title = 'Distribución de cargos del seguro')

#### Distribución de cargos por sexo, región y por condición fumadora

Es interesante compararar el comportamiento de los cargos por sexo, por región y por condición fumadora. A priori, podríamos afirmar que no debería existir diferencias en el comportamiento para hombres y mujeres, mientras que para condiciones como región y condición fumadora si debería existir diferencias. Esto lo podemos ver en las gráficas boxplot pues entre hombres y mujeres, aparentemente no existe cambios en la mediana, aunque los cargos tienen una mayor dispersión para el caso de los hombres. Faltaría ver si existe una diferencia significativa entre las medias de uno y otro grupo.

In [8]:
px.box(df_seguros, x = 'sexo', y = 'cargos', title = 'Distribución de cargos de seguro por sexo')

Por región existe un caso similar al sexo. En la región sureste se nota una mayor dispersión de los gastos siendo el cuartil 3 más alto que en las otras regiones, mientras que las medianas aparentemente son similares. Esto se debe a la gran cantidad de outliers que se encuentran en los datos, lo que muestra la robustez de la mediana ante estos valores.

In [9]:
px.box(df_seguros, x = 'region', y = 'cargos', title = 'Distribución de cargos del seguro por región')

Por último, si comparamos fumadores (1) vs no fumadores (0) encontramos lo esperado, los costos en el seguro se disparan para aquellas personas que suelen fumar.

In [10]:
px.box(df_seguros, x = 'fumador', y = 'cargos', title = 'Distribución de cargos del seguro por fumadores')

#### Distribución de cargos por edad y IMC

Para comparar dos variables continuas es recomendable el uso de gráficos de dispersión. Compararemos a continuación la edad y el IMC vs los cargos del seguro.

En cuanto a la edad, se nota una correlación entre ambas variables, es decir, mientras una crece la otra también lo hace, sin embargo esta correlación no es tan marcada, y además logramos identificar tres tipos de tendencia, una más marcada que las otras.

In [11]:
px.scatter(df_seguros,x = 'edad', y = 'cargos')

Si cruzamos la edad y el cargo del seguro con la condición de fumar de las personas, entonces encontramos que el grupo con cargos más alto, corresponde a los fumadores, mientras que el grupo con cargos menores, y donde la relación entre edad y cargos es más fuerte, es para los no fumadores.

Es decir, la edad tiene un efecto más significativo en los cargos del seguro cuando las personas no fuman.

In [12]:
px.scatter(df_seguros,x = 'edad', y = 'cargos', color = 'fumador')

En cuanto a la dispersión del IMC vs los cargos del seguro podemos identificar un patrón muy marcado, en el cual se observan dos grupos y aparentemente el IMC no tiene correlación con los cargos, en uno de ellos mientras que en el otro si se tiene una correlación muy alta. Indagando un poco más en las variables, nos damos cuenta que estos grupos están marcados por fumadores y no fumadores, lo cual tiene sentido, pues ya habíamos visto que la variable "fumador" está altamente relacionada con los costos.

In [13]:
px.scatter(df_seguros,x = 'imc', y = 'cargos', title = 'Dispersión de los cargos del seguro por IMC')

In [14]:
px.scatter(df_seguros,x = 'imc', y = 'cargos', color='fumador', title = 'Dispersión de los cargos del seguro por IMC y condición de fumador')

#### Conclusiones:

1. Para una persona que fuma, el costo del seguro es "significativamente" mayor que para personas que no fuman
2. Las variables región y sexo, aparentemente no tienen efecto en el costo del seguro. Al menos, en el cuantil 50 (de la mediana hacia abajo)
3. Para personas que no fuman, la relación entre edad y costo del seguro es más fuerte que para personas que fuman
4. Para personas que no fuman, la relación entre IMC y costo del seguro es menos fuerte que para personas que fuman

### Análisis de regresión

Un análisis de regresión nos sirve para:

1. Predecir una variable en función de otras a través de un modelo (comunmente lineal)
2. Observar el comportamiento de un conjunto de variables en el comportamiento de otra

Supongamos que tenemos una variable de interés $y$ y un conjunto de variables relacionadas con $y$ que son $\{x_1, x_2, ..., x_n\}$

El modelo de regresión lineal es el siguiente:

$$y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_n x_n + \varepsilon$$

En este modelo, los valores $\beta_0, ..., \beta_n$ son los parámetros de la regresión, y $\varepsilon$ es el término de error del modelo.

Por ejemplo, supongamos que queremos obtener el ingreso de una familia, y conocemos el número de integrantes y la edad promedio de los integrantes. Es posible ver que el ingreso lo podemos calcular en función de la edad promedio y del número de integrantes. En este caso, la variable $y$ es el ingreso de la familia, y las variables $x_1, x_2$ es el número de integrantes y la edad promedio.
$\newline$
En el ejemplo, el modelo de regresión sería:

$$y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \varepsilon$$

Los parámetros asociados a las variables, nos representan el efecto de la variable explicativa (las x's) hacia la variable respuesta (y). Si son valores positivos, decimos que la variable explicativamente es directamente proporcional a la variable respuesta y si son valores negativos, decimos que es inversamente proporcional a la variable respuesta.

El parámetro $\beta_0$ se le conoce como intercepto o constante del modelo y es un efecto implícito en caso de que las variables no fueran significativas.

El objetivo de la técnica de la regresión es obtener (estimar) los valores de los parámetros.

In [15]:
df_seguros.head(1)

,edad,sexo,imc,hijos,fumador,region,cargos
0,19,femenino,27.9,0,1,suroeste,16884.924


Realizaremos un análisis de regresión para explicar la variable **cargos** en función de las demás variables. Para ello, ajustamos el siguiente modelo:

$$cargos = \beta_0 + \beta_1 \text{edad} + \beta_2 sexo + \beta_3 imc + \beta_4 hijos + \beta_5 fumador + \beta_6 region + \varepsilon$$

En donde $\beta_0$ representa el intercepto o el coeficiente constante, mientras que $\beta$ es el vector de parámetros correspondiente a las características $X$, y $\varepsilon$ es el error en el modelo.

Cuando en una regresión se tienen variables categóricas, estás se deben codificar, es decir, asignarle un número a cada categoría.

In [16]:
## Codificamos las variables categoricas
df_seguros["sexo"] = df_seguros["sexo"].map({"femenino":0, "masculino":1})
df_seguros["region"] = df_seguros["region"].map({"suroeste":0, "sureste":1, "noroeste":2, "noreste":3})

Cuando aplicamos un algoritmo de regresión, es común partir los datos en dos: el conjunto de entrenamiento que nos sirve para entrenar el modelo (estimar los parámetros) y el conjunto de prueba (calcular los errores)

Realizamos la regresión con el siguiente código:

In [17]:
# División de los datos en train y test
# ==============================================================================
X = df_seguros[['edad', 'sexo', 'imc','hijos','fumador','region']]
y = df_seguros['cargos']

X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y.values.reshape(-1,1),
                                        train_size   = 0.8,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

In [18]:
# Creación del modelo utilizando matrices
# ==============================================================================
# A la matriz de predictores se le tiene que añadir una columna de 1s para el
# intercept del modelo
X_train = sm.add_constant(X_train, prepend=True)
modelo  = sm.OLS(endog=y_train, exog=X_train,)
modelo  = modelo.fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.749
Model:                            OLS   Adj. R-squared:                  0.747
Method:                 Least Squares   F-statistic:                     527.9
Date:                Fri, 26 Jan 2024   Prob (F-statistic):          1.20e-314
Time:                        11:04:31   Log-Likelihood:                -10841.
No. Observations:                1070   AIC:                         2.170e+04
Df Residuals:                    1063   BIC:                         2.173e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.207e+04   1162.771    -10.378      0.0

En los resultados obtenidos vale la pena observar lo siguiente:

* R-squared: Debe ser un valor cercano a 1, nos indica la variabilidad de la variable respuesta cubierta por los datos.

* Prob (F-statistic): Es el p-value del siguiente juego de hipótesis: La hipótesis nula es que ninguna variable explicativa es significativa, y la hipótesis alternativa es que al menos una variable explicativa es significativa.

* El p-value de cada parámetro: Si es menor a 0.05 decimos que la variable correspondiente tiene un efecto significativo en la variable respuesta, de otro modo, decimos que la variable correspondiente no tiene un efecto significativo, es decir, no afecta en nada al valor de la variable respuesta.

* El valor de cada parámetro: Nos indica cómo varía el valor de la variable respuesta, en caso de incrementar una unidad la variable correspondiente y manteniendo las demás variables constantes.

En el ejercicio, tanto la variable sexo y la variable región no tienen efectos significativos en el costo del seguro, por lo tanto las podemos omitir del modelo de regresión. Entonces el modelo a ajustar es el siguiente:

$$cargos = \beta_0 + \beta_1 \text{edad} + \beta_2 imc + \beta_3 hijos + \beta_4 fumador + \varepsilon$$

In [19]:
# Creación del modelo utilizando matrices
# ==============================================================================
X_train = X_train.drop(columns = ['sexo','region'])
X_test  = X_test.drop(columns = ['sexo','region'])

# A la matriz de predictores se le tiene que añadir una columna de 1s para el
# intercept del modelo
X_train = sm.add_constant(X_train, prepend=True)
modelo  = sm.OLS(endog=y_train, exog=X_train,)
modelo  = modelo.fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.748
Model:                            OLS   Adj. R-squared:                  0.747
Method:                 Least Squares   F-statistic:                     789.5
Date:                Fri, 26 Jan 2024   Prob (F-statistic):          1.07e-316
Time:                        11:04:32   Log-Likelihood:                -10843.
No. Observations:                1070   AIC:                         2.170e+04
Df Residuals:                    1065   BIC:                         2.172e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.145e+04   1079.990    -10.598      0.0

La ecuación de regresión ajustada es la siguiente:

$$cargos = -1145 + 247.0704\times\text{edad} + 311.2904\times\text{imc} + 538.1388\times\text{hijos} + 2419\times{fumador}$$

Podemos calcular los intervalos de confianza de cada uno de los parámetros, y nos indican el rango de valores dentro de los cuales caerían los coeficientes con una probabilidad del 95%.

In [20]:
# Intervalos de confianza para los coeficientes del modelo
# ==============================================================================
intervalos_ci = modelo.conf_int(alpha=0.05)
intervalos_ci.columns = ['2.5%', '97.5%']
intervalos_ci

,2.5%,97.5%
const,-13565.195851,-9326.896152
edad,220.813693,273.327124
imc,249.972886,372.607995
hijos,235.760135,840.517561
fumador,23277.107481,25103.748998


Podemos calcular predicciones del costo de los seguros si conocemos los valores de las variables explicativas. Por ejemplo, para una persona de 25 años, con IMC igual a 25.70, sin hijos y es fumador, la estimación de su costo de seguro sería el siguiente. Para una persona con estas características, el costo del seguro estimado (proyectado) es de 26,921 pesos, la estimación tiene un error de 502.91676, y cae con un 95% de probabilidad en el intervalo de confianza (25934.48, 27908.12)

In [21]:
# Predicciones con intervalo de confianza
# ==============================================================================

X_new = pd.DataFrame({'const':[1], 'edad':[25], 'imc':[25.70], 'hijos':[0],'fumador':[1]})
predicciones = modelo.get_prediction(exog = X_new).summary_frame(alpha=0.05)
predicciones

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,26921.306773,502.91676,25934.486544,27908.127001,14898.396074,38944.217472


In [22]:
# Error de test del modelo
# ==============================================================================
X_test = sm.add_constant(X_test, prepend=True)
predicciones = modelo.predict(exog = X_test)
rmse = mean_squared_error(
        y_true  = y_test,
        y_pred  = predicciones,
        squared = False
       )
print("")
print(f"El error (rmse) de test es: {rmse}")


El error (rmse) de test es: 5937.234225756581


In [23]:
np.sqrt(rmse)

77.05345044679427

La fórmula del error cuadrado medio es la siguiente:

$$RMSE = \frac{\sum{(y_i - \hat{y}_i)^2}}{n} $$

### Prueba de medias

Una técnica muy utilizada es la prueba de hipótesis. En ella, ponemos a prueba dos hipótesis, la hipótesis nula y la hipótesis alternativa.

En este ejercicio, queremos probar si existe diferencia entre los costos del seguro para hombres y mujeres. Entonces, si asumimos que $\mu_h$ es el costo promedio en los hombres y $\mu_m$ es el costo promedio en las mujeres, entonces la hipótesis nula a probar es:

$$H_0: \mu_h = \mu_m$$

Contra la alternativa:

$$H_1: \mu_h \neq \mu_m$$

La prueba utilizada es una prueba T que utiliza un estadístico de prueba dado por:

$$T = \frac{\bar{x_h}-\bar{x_m}}{\frac{S}{\sqrt{n}}}$$

La hipótesis nula se rechaza si el valor T es muy grande, o en específico, si es más grande que un valor teórico t-student con n-1 grados de libertad y una significancia $\alpha$

En python o cualquier paquete estadístico realizan las pruebas de hipótesis y dan como resultado el p-value, y con ello rechazamos la hipótesis nula para valores pequeños (menores a 0.05) y no rechazamos para valores grandes (mayores a 0.05)

In [24]:
pg.ttest(
    x           = df_seguros[df_seguros['sexo']==1]['cargos'],
    y           = df_seguros[df_seguros['sexo']==0]['cargos'],
    alternative = 'two-sided',
    paired      = True,
    correction  = False
)

C:\Users\jose.lozas\AppData\Roaming\Python\Python39\site-packages\pingouin\parametric.py:228: UserWarning:

x and y have unequal sizes. Switching to paired == False. Check your data.



,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,2.097547,1336,two-sided,0.036133,"[89.81, 2684.53]",0.114693,0.539,0.554144


Tenemos como resultado de realizar la prueba de hipótesis que el p-value es menor a 0.05, es decir, tenemos evidencia para rechazar la hipótesis nula, es decir, hay elementos para decir que el costo medio de los hombres es diferente al costo medio de las mujeres. Esto, si bien contradice un poco el efecto no significativo de la regresión, debemos entender que el p-value es 0.03 (un valor no tan pequeño), y además para que la prueba sea efectiva se deben de cumplir ciertos supuestos, como el de normalidad (que ya vimos que el costo de los seguros no es normal)

Además, en el boxplot vimos que el segundo cuartil (50%) de hombres y mujeres tienen el mismo comportamiento, la diferencia se da después del 75%. Esto se debe a que la media es sensible a valores extremos (y los hombres ya vimos que tienen costos más altos en los extremos), por lo cual en el caso de hombres y mujeres se sugiere mejor utilizar la media.